In [1]:
#read files
import pandas as pd
df = pd.read_csv('all_stay_spec_row_ids.csv', sep='\t', encoding='utf-8')
df_radi = pd.read_csv('all_stay_with_row_ids_final.csv', encoding='utf-8') 
note = pd.read_csv("NOTEEVENTS.csv")

/home/shan/miniconda3/envs/mimic3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Loop through all the CSV per episode, calculate length of stay, eliminate time less than 24. Put in bucket. Then add into Json dict

In [6]:
#function calculating time difference in days
from datetime import datetime

def time_diff (intime, outtime):
    exout = datetime.strptime(outtime, "%Y-%m-%d %H:%M:%S")
    exin = datetime.strptime(intime, "%Y-%m-%d %H:%M:%S")
    difference = exout - exin 
    duration_in_s = difference.total_seconds() 
    return duration_in_s/86400

In [7]:
#label: 3+ 3- 7+ 7-

In [8]:
def label_gen(los_day):
    if los_day < 3:
        return {'3-':'Y','3+':'N','7-':'Y','7+':'N'}
    elif los_day >= 3 and los_day <= 7:
        return {'3-':'N','3+':'Y','7-':'Y','7+':'N'}
    else:
        return {'3-':'N','3+':'Y','7-':'N','7+':'Y'}

In [ ]:
def remove_newline(review):
    review = review.replace('&#039;', "'")
    review = review.replace('\n', ' <cr> ')
    review = review.replace('\r', ' <cr> ')
    review = review.replace('\t', ' ')
    return review

In [10]:
import ast
from tqdm.contrib import tzip
def time_and_text_extract(df):
    re_dict = {}
    len_d = [i for i in range(len(df))]
    for i in tzip(zip(len_d, df.INTIME, df.OUTTIME)):
        i, intime, outtime = i[0][0], i[0][1], i[0][2]
        try:
            los = time_diff(intime, outtime)
        except:
            print(intime,outtime)
        if los>1:
            label = label_gen(los)
            ids = ast.literal_eval(df.row_ids_radi.iloc[i])
            for id in ids:
                text = remove_newline(note.loc[note['ROW_ID'] == id]['TEXT'].any())
                re_dict[str(id)] = {'text':text, 'labels':label}
    return re_dict

In [11]:
radi_dict = time_and_text_extract(df_radi)

0it [00:00, ?it/s]

2147-01-18 19:11:00 nan


In [ ]:
len(radi_dict)

243493

In [65]:
from typing import List
from sklearn.model_selection import train_test_split

def split_data(ratio: List[int], data):
### train, dev, test sum in softmax manner
    s = pd.Series(data)
    training_data, dev_test_data  = [i.to_dict() for i in train_test_split(s, train_size=ratio[0])]
    dev_test_data = pd.Series(dev_test_data)
    dev_data, test_data  = [i.to_dict() for i in train_test_split(dev_test_data, train_size=ratio[1]/(ratio[1]+ratio[2]))]    
    return training_data, dev_data, test_data

[813441, 813564, 812958, 813408, 813232, 813464, 813503, 813681, 813345]

In [ ]:
training_data, dev_data, test_data = split_data([0.7,0.1,0.2], radi_dict)
print(len(training_data), len(dev_data),len(test_data))

In [12]:
import json

with open('training.json', 'w', encoding ='utf8') as json_file:
    json.dump(train, json_file, ensure_ascii = True)

with open('dev.json', 'w', encoding ='utf8') as json_file:
    json.dump(dev, json_file, ensure_ascii = True)

with open('test.json', 'w', encoding ='utf8') as json_file:
    json.dump(test, json_file, ensure_ascii = True)